In [5]:
import pandas as pd
import requests
import re
from tqdm import tqdm

# 카카오 REST API 키 설정
KAKAO_API_KEY = "f32e9311c06e941799af2d38345741de"
HEADERS = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}

# 파일 경로
input_file = "KIC_data.csv"
output_file = "KIC_좌표.csv"

# 데이터 로드 (인코딩 수정)
df = pd.read_csv(input_file, encoding='cp949')  # euc-kr 또는 cp949

address_col = df.iloc[:, 16]  # 17번째 열 (지번 주소)
name_col = df.iloc[:, 2]  # 3번째 열 (이름)

# 결과 저장 리스트
results = []

# 📌 주소 전처리 함수
def clean_address(address):
    # '외 n필지' 제거
    address = re.sub(r"\s*외\s*\d+필지", "", address)
    
    # 블록 제거 (예: 31블록, 3블록)
    address = re.sub(r"\d+블록", "", address)
    
    # '60-1B 5-2L'와 같은 패턴 제거
    address = re.sub(r"\d+[A-Z]\s*\d+-\d+[A-Z]", "", address)
    
    # 여러 공백 하나로
    address = re.sub(r"\s+", " ", address).strip()
    
    return address

# 좌표 변환 함수
def get_coordinates(address):
    url = "https://dapi.kakao.com/v2/local/search/address.json"
    params = {"query": address}
    response = requests.get(url, headers=HEADERS, params=params)
    if response.status_code == 200:
        data = response.json()
        if data["documents"]:
            coords = data["documents"][0]["address"]
            return coords["y"], coords["x"]
        else:
            print(f"❌ 주소를 찾을 수 없음: {address}")
            return None, None
    else:
        print(f"❌ API 오류: {response.status_code} - {response.text}")
        return None, None

# 좌표 변환 및 결과 저장
for name, address in tqdm(zip(name_col, address_col), total=len(address_col)):
    cleaned_address = clean_address(address)
    lat, lon = get_coordinates(cleaned_address)
    if lat and lon:
        results.append([name, lat, lon])

# 데이터프레임 생성 및 저장
result_df = pd.DataFrame(results, columns=["센터명", "위도", "경도"])
result_df.to_csv(output_file, index=False, encoding="utf-8-sig")
print(f"✅ 완료! 저장된 파일: {output_file}")


  1%|          | 16/1539 [00:01<01:59, 12.75it/s]

❌ 주소를 찾을 수 없음: 강원특별자치도 원주시 반곡동 1922-1번지
❌ 주소를 찾을 수 없음: 강원특별자치도 원주시 지정면 가곡리 0번지 원주기업도시내 기업 1-1


  2%|▏         | 26/1539 [00:02<01:57, 12.89it/s]

❌ 주소를 찾을 수 없음: 경기도 고양시 덕양구 덕은동 319-5 고양덕은지구 업무용지 1BL
❌ 주소를 찾을 수 없음: 경기도 고양시 덕양구 덕은동 업무시설용지
❌ 주소를 찾을 수 없음: 경기도 고양시 덕양구 덕은동 업무시설용지 8,


  2%|▏         | 36/1539 [00:02<02:05, 11.93it/s]

...

❌ 주소를 찾을 수 없음: 경기도 안양시 만안구 박달동 617-12 장, 617-13도, 617-35임, 산30-4임, 산30-8임


 35%|███▌      | 541/1539 [00:40<00:57, 17.46it/s]

❌ 주소를 찾을 수 없음: 경기도 용인시 기흥구 구갈동 259-1번지 기흥ICT밸리 도시첨단산업단지 B블럭
❌ 주소를 찾을 수 없음: 경기도 용인시 기흥구 영덕동 784


 40%|████      | 618/1539 [00:46<01:09, 13.23it/s]


KeyboardInterrupt: 